# Для того, чтобы установить Invoke Ai, запустите код в каждой из форм.
Для запуска сервера потребуется аккаунт на [ngrok](https://dashboard.ngrok.com/login)

# To use Invoke Ai, launch the code in every form below.
You will need an account on [ngrok](https://dashboard.ngrok.com/login)

**[Repo](https://github.com/peaashmeter/invoke-ai-gui-colab)**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Во время этого этапа Colab крашится, это нормально
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title Подготовка репозитория Invoke Ai { display-mode: "form" }
%cd /home

!git clone https://github.com/invoke-ai/InvokeAI.git
%cd InvokeAI

!ln -sf environments-and-requirements/environment-lin-cuda.yml environment.yml
!ls -la

%cd ..

!git clone https://github.com/peaashmeter/invoke-ai-gui-colab.git
%cd invoke-ai-gui-colab

!cp cross_attention_control.py ../InvokeAI/ldm/models/diffusion/cross_attention_control.py

In [ ]:
#@title Установка зависимостей, в норме занимает ~10 минут { display-mode: "form" }
%cd ../InvokeAI
!pip install pyngrok --quiet
!conda env update
!source activate invokeai ; python scripts/configure_invokeai.py --yes

%cp ../invoke-ai-gui-colab/models.yaml /root/invokeai/configs 



In [ ]:
#@title Установка модели Anything-V3.0 (требуется 5.1 Гб места на гугл-диске) { display-mode: "form" }
import os


os.system('cd /')
os.system('mkdir -p /root/invokeai/models/ldm/stable-diffusion-v1')

if os.path.exists('/content/drive/MyDrive/models/Anything-V3.0-pruned.ckpt'):
    None
else:
    print('Производится скачивание модели Anything-V3.0')
    os.system('mkdir -p /content/drive/MyDrive/models/')
    os.system('wget -O /content/drive/MyDrive/models/Anything-V3.0-pruned.ckpt https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0-pruned.ckpt')

if os.path.exists('/content/drive/MyDrive/models/Anything-V3.0.vae.pt'):
  None
else:
  print('Производится скачивание Anything-VAE')
  os.system('wget -O /content/drive/MyDrive/models/Anything-V3.0.vae.pt https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0.vae.pt')

if os.path.exists('/content/drive/MyDrive/models/vae-ft-mse-840000-ema-pruned.ckpt'):
  None
else:
  print('Производится скачивание SD-VAE')
  os.system('wget -O /content/drive/MyDrive/models/vae-ft-mse-840000-ema-pruned.ckpt https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt')

if not os.path.exists('/content/drive/MyDrive/models/Anything-V3.0-pruned.ckpt') or not os.path.exists('/content/drive/MyDrive/models/vae-ft-mse-840000-ema-pruned.ckpt') or not os.path.exists('/content/drive/MyDrive/models/Anything-V3.0.vae.pt'):
    print('Произошла ошибка, попробуйте еще раз\nUnexpected error')
else:
    print('Make sure you accept the terms at https://huggingface.co/Linaqruf/anything-v3.0 and https://huggingface.co/stabilityai/sd-vae-ft-mse-original')


%mkdir -p /root/invokeai/models/ldm/stable-diffusion-v1/
!ln -s  /content/drive/MyDrive/models/Anything-V3.0-pruned.ckpt /root/invokeai/models/ldm/stable-diffusion-v1/Anything-V3.0-pruned.ckpt
!ls -l /root/invokeai/models/ldm/stable-diffusion-v1/Anything-V3.0-pruned.ckpt

!ln -s  /content/drive/MyDrive/models/vae-ft-mse-840000-ema-pruned.ckpt /root/invokeai/models/ldm/stable-diffusion-v1/vae-ft-mse-840000-ema-pruned.ckpt
!ls -l /root/invokeai/models/ldm/stable-diffusion-v1/vae-ft-mse-840000-ema-pruned.ckpt

!ln -s  /content/drive/MyDrive/models/Anything-V3.0.vae.pt /root/invokeai/models/ldm/stable-diffusion-v1/Anything-V3.0.vae.pt
!ls -l /root/invokeai/models/ldm/stable-diffusion-v1/Anything-V3.0.vae.pt

In [ ]:
#@title Запуск сервера { display-mode: "both" }
#@markdown Вставьте токен с [ngrok](https://dashboard.ngrok.com/get-started/your-authtoken) в поле перед выполнением кода.

ngrok_token = "" #@param {type:"string"}
nsfw_checker = 0 #@param {type:"slider", min:0, max:1, step:1}
used_vae = "Anything-V3.0.vae.pt" #@param ["Anything-V3.0.vae.pt", "vae-ft-mse-840000-ema-pruned"]

import os
from pyngrok import ngrok

ngrok.kill()
ngrok.set_auth_token(ngrok_token)
public_url = ngrok.connect(9090).public_url
print(f'Invoke Ai public url: {public_url}')

%cd /home/InvokeAI

model_name = "Anything_v3_vaefixed"
if used_vae == "vae-ft-mse-840000-ema-pruned":
    model_name = "Anything_v3_default"


if nsfw_checker:
    !source activate invokeai ; python scripts/invoke.py --web --no-nsfw_checker --model $model_name
else:
    !source activate invokeai ; python scripts/invoke.py --web --no-nsfw_checker --model $model_name

# После запуска сервера нужно перейти по первой ссылке из вывода (Invoke Ai public url)
Если интерфейс не прогрузился, надо перезагрузить страницу
---

